# Tree-based Models, with Clusters

## Library Imports

In [1]:
# Necessary code to import our helper functions
import sys
sys.path.append("../..")

In [2]:
# Library imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from matplotlib import pyplot as plt
import xgboost as xgb
from sklearn.metrics import mean_absolute_percentage_error
from Common_Functions import data_split, add_unique_identifier, data_cleaning, hospital_data_agg

C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


## Data Import

In [3]:
data = pd.read_csv("../../Data_Files/JnJ_Files/priv_mcare_f_pay_2022Oct18.csv")
hospital_data = pd.read_csv("../../Data_Files/JnJ_Files/Hospital_Master_Sheet.csv")

## Add cluster labels

In [4]:
clusters = pd.read_csv("../../Kmeans_clusters/clusters_only_using_NormCost.csv")

In [5]:
data = data.merge(clusters, how="left",on="group").dropna(subset = "cluster")
data["cluster"] = data["cluster"].astype(int)

## Model Parameters

In [6]:
COUNT_THRESH = 34
RDM_SEED = 123
TRAIN_TEST_PROPORTION = 0.8

## Data Transformation

### One-Hot Categorical Encoding and Dropping NAs

In [7]:
data.drop(columns = ['CBSA_NAME'], inplace=True)
data = data_cleaning(data)

### Hospital Data

In [8]:
hospital_msa = hospital_data_agg(hospital_data)

display(hospital_msa)

,msa,Hospitals,PctTeaching,PctLargeHospital,Urban,PctPrivate
0,1,64,0.062500,0.000000,0.0,0.484375
1,2,13,0.076923,0.000000,0.0,0.153846
2,3,18,0.111111,0.000000,0.0,0.444444
3,4,71,0.098592,0.028169,0.0,0.591549
4,5,42,0.023810,0.000000,0.0,0.214286
...,...,...,...,...,...,...
479,99944,4,0.000000,0.000000,1.0,0.000000
480,99945,1,0.000000,0.000000,1.0,0.000000
481,99949,1,0.000000,0.000000,1.0,1.000000
482,99951,2,0.500000,0.000000,1.0,0.000000


### Data Split

In [9]:
working_set, predict_set = data_split(data, count_thresh = COUNT_THRESH)

In [10]:
model_data = working_set.merge(hospital_msa,how="left", on="msa").drop(columns=["priv_pay_mean",
                                                                                "priv_pay_iqr",
                                                                                "mcare_pay_mean",
                                                                                "mcare_pay_sd",
                                                                                "Urban",
                                                                                "msa"])
predict_data = predict_set.merge(hospital_msa,how="left", on="msa").drop(columns=["priv_pay_mean",
                                                                                  "priv_pay_iqr",
                                                                                  "mcare_pay_mean",
                                                                                  "mcare_pay_sd",
                                                                                  "Urban",
                                                                                  "msa"])

In [11]:
display(model_data)

,year,priv_count,priv_pay_median,mcare_count,mcare_los,mcare_pay_median,lon,lat,cluster,site_ASC,...,State_Vermont,State_Virginia,State_Washington,State_West Virginia,State_Wisconsin,State_Wyoming,Hospitals,PctTeaching,PctLargeHospital,PctPrivate
0,2018,35.0,10083.34,88.0,0.0,4376.17,-83.743038,42.280826,0,0,...,0,0,0,0,0,0,6,0.500000,0.333333,0.500000
1,2019,35.0,9076.20,56.0,0.0,5985.04,-105.270546,40.014986,0,0,...,0,0,0,0,0,0,5,0.000000,0.000000,0.600000
2,2019,35.0,17251.33,121.0,0.0,4386.17,-111.891047,40.760779,0,0,...,0,0,0,0,0,0,18,0.388889,0.055556,0.777778
3,2020,35.0,11520.39,20.0,0.0,3896.76,-91.665623,41.977880,0,0,...,0,0,0,0,0,0,6,0.166667,0.166667,0.333333
4,2020,35.0,16731.80,97.0,0.0,3719.49,-78.928824,33.919657,0,0,...,0,0,0,0,0,0,7,0.142857,0.000000,0.571429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4518,2018,1674.0,8942.50,1112.0,0.0,3749.44,-95.622552,29.598443,0,0,...,0,0,0,0,0,0,181,0.088398,0.060773,0.823204
4519,2018,1843.0,14929.63,2576.0,0.0,4331.08,-96.920913,32.707875,0,0,...,0,0,0,0,0,0,114,0.105263,0.052632,0.807018
4520,2018,1900.0,8746.36,1084.0,0.0,3647.39,-84.294090,34.075376,0,0,...,0,0,0,0,0,0,80,0.162500,0.050000,0.725000
4521,2018,1919.0,10701.58,4248.0,0.0,5153.90,-74.005954,40.712776,0,0,...,0,0,0,0,0,0,143,0.552448,0.230769,0.643357


## Split Model Data by Cluster

In [12]:
cluster_data_list = []
for cluster_label in model_data["cluster"].unique():
    cluster_data_list.append(model_data[model_data["cluster"] == cluster_label])

## Run XGBoost model

In [13]:
train_mapes = []
train_sizes = []
test_mapes = []
test_sizes = []

# Train test split
for cluster_dataset in cluster_data_list:

    X_input = cluster_dataset.drop(columns=["priv_pay_median"])
    y_input = cluster_dataset["priv_pay_median"]

    X_train, X_test, y_train, y_test = train_test_split(X_input,
                                                        y_input,
                                                        train_size = TRAIN_TEST_PROPORTION,
                                                        random_state = RDM_SEED)
    # Parameterization
    param_grid = {
        'n_estimators':[100, 150, 200],
        'max_depth':[5,10,15],
        'learning_rate': [0.1,0.2,0.3],
#         'gamma':[0, 0.01,0.1],
    #     'min_child_weight':[0,1],
        'reg_lambda':[0,0.25,0.5,0.75]
    }
    mono = {'site_Inpatient': 1}
    
    # Create, run, and tune (if applicable) model
    xgb_param_tuning_model = xgb.XGBRegressor(monotone_constraints = mono,
#                                               n_estimators = 200,
#                                               max_depth = 10,
                                              gamma = 0,
                                              min_child_weight = 0,
#                                               learning_rate = 0.2,
#                                               reg_lambda = 0.5
                                             )
    xgb_mono_model = GridSearchCV(xgb_param_tuning_model, param_grid, scoring='neg_mean_absolute_percentage_error')
    xgb_mono_model.fit(X_train, y_train)
    
    # Output optimal params (if applicable)
    print(f"Best parameters (if grid search was applied): {xgb_mono_model.best_params_}")
    
    # Predict on train and test data
    y_train_pred_xgb = xgb_mono_model.predict(X_train)
    y_test_pred_xgb = xgb_mono_model.predict(X_test)

    # Store results
    train_sizes.append(len(X_train))
    test_sizes.append(len(X_test))
    train_mapes.append(mean_absolute_percentage_error(y_true=y_train, y_pred=y_train_pred_xgb))
    test_mapes.append(mean_absolute_percentage_error(y_true=y_test, y_pred=y_test_pred_xgb))
    

train_mapes = np.array(train_mapes)
train_sizes = np.array(train_sizes)
test_mapes = np.array(test_mapes)
test_sizes = np.array(test_sizes)

# Output results?
print(f"Monotonic XGBoost with Threshold >{COUNT_THRESH} claims for training set:")
print(f"Train MAPEs: {train_mapes}")
print(f"Train sizes: {train_sizes}")
print(f"Test MAPEs: {test_mapes}")
print(f"Test sizes: {test_sizes}")
print(f"Total train MAPE: {((train_mapes * train_sizes) / (train_sizes.sum())).sum()}")
print(f"Total test MAPE: {((test_mapes * test_sizes) / (test_sizes.sum())).sum()}")

C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future

C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future

C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future

C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future

C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future

...

C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future

C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future

C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future

C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


Best parameters (if grid search was applied): {'learning_rate': 0.3, 'max_depth': 10, 'n_estimators': 100, 'reg_lambda': 0.25}
Monotonic XGBoost with Threshold >34 claims for training set:
Train MAPEs: [7.48567970e-02 8.20703233e-04 8.31778538e-08]
Train sizes: [2172 1278  168]
Test MAPEs: [0.16823214 0.14511939 0.15663505]
Test sizes: [543 320  42]
Total train MAPE: 0.04522881030739373
Total test MAPE: 0.15952147011815115
